#Demo de Mars Explorer simulando la arquitectura Subsumption.
- Objetivo : Explorar un planeta distante y en particular recolectar muestras de un cierto tipo de roca.
- Problema: La localización de las rocas no se conoce, pero se sabe que ellas tienden a estar agrupadas en regiones cercanas

In [1]:
#@title Cargar Librerías

from random import randint
import copy

print("Librerías cargadas.")

Librerías cargadas.


In [43]:
#@title Definir Mapa al Azar
limiteMaximo = 10 #@param{"type":"integer"}
cantObstaculos = 30 #@param{"type":"integer"}
cantMuestras = 15 #@param{"type":"integer"}
if limiteMaximo<10:
  limiteMaximo = 10
if cantObstaculos<1:
  cantObstaculos = 1
if cantMuestras<1:
  cantMuestras = 1

# genera obstaculos
ObstaculosPos = []
for _ in range(30):
  ObstaculosPos.append(  [ randint(0, limiteMaximo),  randint(0, limiteMaximo) ] )
print("Obstaculos en ", ObstaculosPos)

# genera muestras
MuestrasPos = []
for _ in range(15):
  auxPos = [ randint(0, limiteMaximo),  randint(0, limiteMaximo) ]
  if auxPos not in ObstaculosPos:
    MuestrasPos.append( auxPos)
print("Muestras en ", MuestrasPos )

Obstaculos en  [[8, 1], [8, 3], [3, 7], [3, 7], [3, 8], [8, 10], [2, 9], [4, 4], [2, 4], [4, 4], [1, 9], [10, 1], [5, 5], [2, 7], [2, 4], [8, 9], [10, 5], [4, 3], [3, 0], [2, 9], [4, 1], [1, 0], [7, 5], [2, 1], [1, 6], [8, 2], [10, 6], [1, 5], [4, 8], [3, 4]]
Muestras en  [[6, 10], [0, 9], [10, 2], [9, 3], [9, 6], [7, 2], [9, 2], [7, 2], [3, 9], [2, 0], [10, 9], [1, 2], [10, 4], [7, 8]]


In [44]:
#@title Definir clase MarsExplorer

POS_NADA = "-"
POS_BASE = "NAVE"
POS_MUESTRA = "MUESTRA"
POS_OBSTACULO = "OBSTACULO"

class MarsExplorer(object):

    def __init__(self, posInicial, limiteMaximo, obstaculosPos, muestrasPos):
      self.posActual = copy.deepcopy(posInicial)
      self.posBase = copy.deepcopy(posInicial)
      self.llevaMuestra = False
      self._limiteMaximo = limiteMaximo
      self._obstaculosPos = obstaculosPos
      self._muestrasPos = muestrasPos

    def determinaMov(self, posRegresar=None):
      # nueva posición
      nPos = copy.deepcopy(self.posActual)
      if posRegresar is None:
          # cambia posición al azar
          for i in range(len(nPos)):
            if (nPos[i]==0):
               nPos[i] += 1
            elif (nPos[i]==self._limiteMaximo):
              nPos[i] -= 1
            else:
              rnd = randint(-1, 1)
              if (i == 0) and (rnd==0):
                nPos[i] += 1
              else:
                nPos[i] += rnd
      else:
          # regresa a la pos indicada
          for i in range(len(nPos)):
            if (nPos[i]>0) and (nPos[i]<self._limiteMaximo):
              if posRegresar[i]>nPos[i]:
                nPos[i] += 1
              elif posRegresar[i]<nPos[i]:
                nPos[i] -= 1
      # devuelve nueva posición
      return nPos

    def distanciaBase(self, pos):
      # calcula distancia entre dos puntos
      dist = 0.0
      for p, b in zip(pos, self.posBase):
        dist += (p - b)**2
      return dist**0.5

    def evaluarPosicion(self, nPos):
      # determina si hay algo en la posicion usando random
      if self.distanciaBase(nPos)<1:
        # está en base
        return POS_BASE
      elif nPos in self._obstaculosPos:
        # hay obstaculo
        return POS_OBSTACULO
      elif nPos in self._muestrasPos:
        # hay muestra
        return POS_MUESTRA
      else:
        return POS_NADA

    def analizarMuestra(self, posNueva):
      # determina si es interesante
      # si la 1er posición es par
      return (randint(1, 2)==1)

    def tomarMuestra(self, pos):
      # toma la muestra
      self.llevaMuestra = True
      # elimina la muestra porque fue tomada
      self._muestrasPos.remove( pos )

    def dejarMuestra(self):
      # deja la muestra
      self.llevaMuestra = False

    def reglasC0(self, posNueva, posEval):
        ##C0 - Evitar obstáculos
        if posEval == POS_OBSTACULO:
          # determina nueva posición al azar
          # descarta posición (no la actualiza)
          return [True, False, None, "C0: Evita OBSTACULO"]
        else:
           return [False, False, None, None]

    def reglasC1(self, posNueva, posEval):
        ##C1 - Dejar muestras en la nave
        if self.llevaMuestra and (posEval == POS_BASE):
          # deja muestra
          self.dejarMuestra()
          return [True, True, None, "C1: Deja MUESTRA en NAVE"]
        else:
          return [False, False,  None, None]

    def reglasC2(self, posNueva, posEval):
        ##C2 - Llevar muestras a la nave
        if self.llevaMuestra and (posEval != POS_BASE):
          # determina nueva posición para regresar nave
          return [True, True, self.posBase, "C2: Llevando MUESTRA a NAVE"]
        else:
          return [False, False, None, None]

    def reglasC3y4(self, posNueva, posEval):
        ##C4 - Identificar muestras
        if not(self.llevaMuestra) and (posEval == POS_MUESTRA):
          # analiza la muestra
          detectaMuestraInt = self.analizarMuestra(posNueva)
          strDesc = "C4: Identifica MUESTRA ( interesante="+str(detectaMuestraInt)+")"
          # si interesante la toma, sino la descarta
          if detectaMuestraInt:
            ##C3 - Recolectar muestras
            strDesc = strDesc + " --> C3: Recolecta MUESTRA"
            # toma muestra
            self.tomarMuestra(posNueva)
            return [True, True, self.posBase, strDesc]
          else:
            return [True, True, None, strDesc]
        else:
          return [False, False,  None, None]

    def reglasC5(self, posNueva, posEval):
        ##C5 - Deambular (moverse al azar)
        # devuelve resultados
        return [True, True, None, "C5- DEAMBULAR"]


    def ejecutar(self, N):
      # define las regla a ejecutar
      reglaComportamiento = [self.reglasC0, self.reglasC1, self.reglasC2, self.reglasC3y4, self.reglasC5]
      # determina posición inicial
      posNueva = self.posBase
      posEval = self.evaluarPosicion(posNueva)
      print(0, "-- Comienza en NAVE ",  self.posBase, "")
      # ejecuta
      for i in range(1, N+1):
        # ejecuta la regla que corresponda
        for regla in reglaComportamiento:
          # ejecuta regla devolviendo:
          #   0: si se cumplieron las condiciones de la regla
          #   1: si se actualiza la posición actual
          #   2: la posición a regresara (si no es None)
          #   3: descripción de la regla
          resRegla = regla(posNueva, posEval)
          # si se ejecuta la regla
          if resRegla[0]:
            # si actualiza posición actual
            if resRegla[1]:
              # actualiza posición
              self.posActual = posNueva

            # determina nueva posición  (al azar o no)
            posNueva = self.determinaMov(resRegla[2])
            # analiza nueva posición
            posEval = self.evaluarPosicion(posNueva)
            # muestra mensaje regla
            if (len(resRegla)>3) and (resRegla[3] is None):
              strDesc = str(regla)
            else:
              strDesc = resRegla[3]
            print("#" +str(i), "--", self.posActual, "--", strDesc, "--> nuevaPos:", posNueva, "(", posEval, ")")
            break


print("clase MarsExplorer definida.")

clase MarsExplorer definida.


In [46]:
#@title Indicar cantidad de ciclos de Ejecución (N)
N = 250 #@param {"type":"integer"}
if N<5:
  N=5

# ejecuta
posIni = [5, 5]
me = MarsExplorer(posIni, limiteMaximo,
                  ObstaculosPos,
                  MuestrasPos)
me.ejecutar(N)


0 -- Comienza en NAVE  [5, 5] 
#1 -- [5, 5] -- C5- DEAMBULAR --> nuevaPos: [4, 6] ( - )
#2 -- [4, 6] -- C5- DEAMBULAR --> nuevaPos: [5, 6] ( - )
#3 -- [5, 6] -- C5- DEAMBULAR --> nuevaPos: [6, 5] ( - )
#4 -- [6, 5] -- C5- DEAMBULAR --> nuevaPos: [7, 6] ( - )
#5 -- [7, 6] -- C5- DEAMBULAR --> nuevaPos: [6, 7] ( - )
#6 -- [6, 7] -- C5- DEAMBULAR --> nuevaPos: [7, 6] ( - )
#7 -- [7, 6] -- C5- DEAMBULAR --> nuevaPos: [8, 6] ( - )
#8 -- [8, 6] -- C5- DEAMBULAR --> nuevaPos: [9, 7] ( - )
#9 -- [9, 7] -- C5- DEAMBULAR --> nuevaPos: [10, 7] ( - )
#10 -- [10, 7] -- C5- DEAMBULAR --> nuevaPos: [9, 6] ( MUESTRA )
#11 -- [9, 6] -- C4: Identifica MUESTRA ( interesante=True) --> C3: Recolecta MUESTRA --> nuevaPos: [8, 5] ( - )
#12 -- [8, 5] -- C2: Llevando MUESTRA a NAVE --> nuevaPos: [7, 5] ( OBSTACULO )
#13 -- [8, 5] -- C0: Evita OBSTACULO --> nuevaPos: [7, 4] ( - )
#14 -- [7, 4] -- C2: Llevando MUESTRA a NAVE --> nuevaPos: [6, 5] ( - )
#15 -- [6, 5] -- C2: Llevando MUESTRA a NAVE --> nuevaPos: [5